###### Imports and Settings

In [1]:
import pandas as pd
import numpy as np
import requests
import pickle
from collections import deque
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 150)
pd.options.mode.chained_assignment = None  # default='warn'

# Comprehensive Plan Data Pull  

In [2]:
#to read in... rb is read bite
with open('api_keys.pkl', 'rb') as keys_file:
        keys_dict_2 = pickle.load(keys_file)

In [3]:
#create a variable that contains your api key
api_key = keys_dict_2['CENSUS']

In [4]:
GNRC = ['111', #Macon
       '161', #Stewart
       '125', #Montgomery
       '083', #Houston
       '085', #Humphreys
       '043', #Dickson
       '021', #Cheatham
       '147', #Robertson
       '165', #Sumner
       '037', #Davidson
       '189', #Wilson
       '169', #Trousdale
       '187', #Williamson
       '149', #Rutherford
       '119'] #Maury
KY = ['003', #Allen
      '213'] #Simpson
qlaces = ['1600000US4702180', #Ashland City: Cheatham
          '1600000US4739660', #Kingston Springs: Cheatham
          '1600000US4757480', #Pegram: Cheatham
          '1600000US4759560', #Pleasant View: Cheatham
          '1600000US4704620', #Belle Meade: Davidson
          '1600000US4705140', #Berry Hill: Davidson
          '1600000US4727020', #Forest Hills: Davidson
          '1600000US4729920', #Goodlettsville: Davidson/Sumner
          '1600000US4752006', #Nashville-Davidson metropolitan government (balance): Davidson
          '1600000US4754780', #Oak Hill: Davidson
          '1600000US4763140', #Ridgetop: Davidson/Robertson
          '1600000US4709880', #Burns: Dickson
          '1600000US4713080', #Charlotte: Dickson
          '1600000US4720620', #Dickson: Dickson
          '1600000US4769080', #Slayden: Dickson
          '1600000US4776860', #Vanleer: Dickson
          '1600000US4779980', #White Bluff: Dickson
          '1600000US4724320', #Erin: Houston
          '1600000US4773460', #Tennessee Ridge: Houston/Stewart
          '1600000US4744840', #McEwen: Humphreys
          '1600000US4752820', #New Johnsonville: Humphreys
          '1600000US4778560', #Waverly: Humphreys
          '1600000US4716540', #Columbia: Maury
          '1600000US4751080', #Mount Pleasant: Maury
          '1600000US4770580', #Spring Hill: Maury/Williamson
          '1600000US4715160', #Clarksville: Montgomery
          '1600000US4700200', #Adams: Robertson
          '1600000US4711980', #Cedar Hill: Robertson
          '1600000US4716980', #Coopertown: Robertson
          '1600000US4718420', #Cross Plains: Robertson
          '1600000US4730960', #Greenbrier: Robertson
          '1600000US4748980', #Millersville: Robertson/Sumner
          '1600000US4760280', #Portland: Robertson/Sumner
          '1600000US4770500', #Springfield: Robertson
          '1600000US4780200', #White House: Robertson/Sumner
          '1600000US4722360', #Eagleville: Rutherford
          '1600000US4741200', #La Vergne: Rutherford
          '1600000US4751560', #Murfreesboro: Rutherford
          '1600000US4769420', #Smyrna: Rutherford
          '1600000US4718820', #Cumberland City: Stewart
          '1600000US4721400', #Dover: Stewart
          '1600000US4728540', #Gallatin: Sumner
          '1600000US4733280', #Hendersonville: Sumner
          '1600000US4779420', #Westmoreland: Sumner
          '1600000US4708280', #Brentwood: Williamson
          '1600000US4725440', #Fairview: Williamson
          '1600000US4727740', #Franklin: Williamson
          '1600000US4753460', #Nolensville: Williamson
          '1600000US4773900', #Thompson's Station: Williamson
          '1600000US4741520', #Lebanon: Wilson
          '1600000US4750780', #Mount Juliet: Wilson
          '1600000US4778320', #Watertown: Wilson
          '1600000US4740160', #Lafayette: Macon
          '1600000US2169114', #Scottsville city, Kentucky
          '1600000US2128918', #Franklin city, Kentucky
          '1600000US4749460'] #Mitchelville city: Sumner

In [30]:
def percent(x, y):
    return (x/y)*100

## Read In Data Guide

### SF1

In [5]:
dataguide = pd.read_csv('../../data guides/ACS1YR.csv', dtype = str)
dataguide['ID'] = dataguide['ID'].astype(int)

In [6]:
dg1 = dataguide[dataguide['ID'].between(1, 46)]
#dg2 = dataguide[dataguide['ID'].between(47, 92)]

In [24]:
url_str= 'https://api.census.gov/data/2021/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in the State of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in the State of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = df.append(kycos).reset_index(drop = True)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = df.append(places).reset_index(drop = True)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(state, ignore_index = True)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(national, ignore_index = True)
savename = df
print('Okay Finished')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3558364531.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(kycos).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3558364531.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3558364531.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3558364531.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(state, ignore_index = True

Okay Finished


C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3558364531.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(national, ignore_index = True)


In [27]:
twentytwentyone = savename
twentytwentyone['Year'] = '2021'
data = twentytwentyone
data.set_index(['NAME', 'GEO_ID', 'Year'])

In [28]:
data.head()

,NAME,GEO_ID,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
0,"Rutherford County, Tennessee",0500000US47149,348345,294979,25077,17428,9996,865,47,149,2021
1,"Sumner County, Tennessee",0500000US47165,198381,171816,8890,9856,7147,672,47,165,2021
2,"Davidson County, Tennessee",0500000US47037,693331,548031,72475,32112,35232,5481,47,037,2021
3,"Maury County, Tennessee",0500000US47119,None,None,None,None,None,None,47,119,2021
4,"Montgomery County, Tennessee",0500000US47125,225185,171736,24547,8914,18236,1752,47,125,2021


In [31]:
#calculations
#geographic mobility
data['GeoMobility:Total'] = data['geo_total_series']
data['GeoMobility:Same House'] = data['geo_samehouse_total']
data['GeoMobility%:Same House'] = percent(data['GeoMobility:Same House'],data['GeoMobility:Total'])
data['GeoMobility:Moved Same County'] = data['geo_movedsamecounty_total']
data['GeoMobility%:Moved Same County'] = percent(data['GeoMobility:Moved Same County'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different County Same State'] = data['geo_moveddifferentcountysamestate_total']
data['GeoMobility%:Moved Different County Same State'] = percent(data['GeoMobility:Moved Different County Same State'],data['GeoMobility:Total'])
data['GeoMobility:Moved Different State'] = data['geo_moveddifferentstate_total']
data['GeoMobility%:Moved Different State'] = percent(data['GeoMobility:Moved Different State'],data['GeoMobility:Total'])
data['GeoMobility:Moved From Abroad'] = data['geo_movedabroad_total']
data['GeoMobility%:Moved From Abroad'] = percent(data['GeoMobility:Moved From Abroad'],data['GeoMobility:Total'])

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [22]:
twentytwentyone.head()

,NAME,GEO_ID,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
0,"Rutherford County, Tennessee",0500000US47149,348345,294979,25077,17428,9996,865,47,149,2021
1,"Sumner County, Tennessee",0500000US47165,198381,171816,8890,9856,7147,672,47,165,2021
2,"Davidson County, Tennessee",0500000US47037,693331,548031,72475,32112,35232,5481,47,037,2021
3,"Maury County, Tennessee",0500000US47119,None,None,None,None,None,None,47,119,2021
4,"Montgomery County, Tennessee",0500000US47125,225185,171736,24547,8914,18236,1752,47,125,2021


In [11]:
url_str= 'https://api.census.gov/data/2019/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in the State of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in the State of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = df.append(kycos).reset_index(drop = True)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = df.append(places).reset_index(drop = True)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(state, ignore_index = True)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(national, ignore_index = True)
savename = df
print('Okay Finished')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3116872924.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(kycos).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3116872924.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3116872924.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3116872924.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(state, ignore_index = True

Okay Finished


C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\3116872924.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(national, ignore_index = True)


In [12]:
twentynineteen = savename
twentynineteen['Year'] = '2019'

In [13]:
url_str= 'https://api.census.gov/data/2018/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in the State of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in the State of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = df.append(kycos).reset_index(drop = True)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = df.append(places).reset_index(drop = True)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(state, ignore_index = True)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(national, ignore_index = True)
savename = df
print('Okay Finished')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2608702143.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(kycos).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2608702143.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2608702143.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2608702143.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(state, ignore_index = True

Okay Finished


C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2608702143.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(national, ignore_index = True)


In [14]:
twentyeighteen = savename
twentyeighteen['Year'] = '2018'

In [15]:
url_str= 'https://api.census.gov/data/2017/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in the State of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in the State of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = df.append(kycos).reset_index(drop = True)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = df.append(places).reset_index(drop = True)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(state, ignore_index = True)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(national, ignore_index = True)
savename = df
print('Okay Finished')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\1663470334.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(kycos).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\1663470334.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\1663470334.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\1663470334.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(state, ignore_index = True

Okay Finished


C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\1663470334.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(national, ignore_index = True)


In [16]:
twentyseventeen = savename
twentyseventeen['Year'] = '2017'

In [17]:
url_str= 'https://api.census.gov/data/2016/acs/acs1?key='+api_key
head1 = 'NAME' 
head2 = 'GEO_ID'
tail_cols1 = 'StateFIPS'
tail_cols2 = 'GeoFIPS'
dataguide = dg1
var_list = list(dataguide['ACS Variable']) #make variables list
var_list = deque(var_list)
var_list.appendleft(head2)
var_list.appendleft(head1)
col_list = list(dataguide['Column Name']) #make columns list
col_list.append(tail_cols1)
col_list.append(tail_cols2)
col_list = deque(col_list)
col_list.appendleft(head2)
col_list.appendleft(head1)
predicates= {} #call for all counties in the State of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:47"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
df = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
df = df.loc[df['GeoFIPS'].isin(GNRC)] #filter for counties in the region
predicates= {} #call for all counties in the State of KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "county:*"
predicates["in"]= "state:21"                                                             
data = requests.get(url_str, params= predicates)                                                                
col_names = col_list
kycos = pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
kycos = kycos.loc[kycos['GeoFIPS'].isin(KY)] #filter for counties of concern in KY
df = df.append(kycos).reset_index(drop = True)
predicates= {} #call for all places in KY
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:21"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places of concern in KY
df = df.append(places).reset_index(drop = True)
predicates= {} #call for all places in TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "place:*"
predicates["in"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
places=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
places=places.loc[places['GEO_ID'].isin(qlaces)] #filter for places in the region
df = df.append(places).reset_index(drop = True)
col_list.remove(tail_cols2) #adjust the column list for different formats of geos
predicates= {} #call for state of TN
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "state:47"
data= requests.get(url_str, params= predicates)
col_names = col_list
state=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
state['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(state, ignore_index = True)
predicates= {} #call for US
get_vars= var_list
predicates["get"]= ",". join(get_vars)
predicates["for"]= "us:*"
data= requests.get(url_str, params= predicates)
col_names = col_list
national=pd.DataFrame(columns=col_names, data=data.json()[1:], dtype=str)
national['GeoFIPS'] = '0' #fill in the blank GeoFIPS column
df = df.append(national, ignore_index = True)
savename = df
print('Okay Finished')

C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2621164383.py:35: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(kycos).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2621164383.py:45: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2621164383.py:55: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(places).reset_index(drop = True)
C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2621164383.py:65: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(state, ignore_index = True

Okay Finished


C:\Users\jmccall\AppData\Local\Temp\ipykernel_2540\2621164383.py:74: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(national, ignore_index = True)


In [18]:
twentysixteen = savename
twentysixteen['Year'] = '2016'

In [19]:
twentysixteen.head()

,NAME,GEO_ID,geo_total_series,geo_samehouse_total,geo_movedsamecounty_total,geo_moveddifferentcountysamestate_total,geo_moveddifferentstate_total,geo_movedabroad_total,StateFIPS,GeoFIPS,Year
0,"Sumner County, Tennessee",0500000US47165,178875,153752,14788,5079,4155,1101,47,165,2016
1,"Davidson County, Tennessee",0500000US47037,675434,558893,61119,21004,29059,5359,47,037,2016
2,"Montgomery County, Tennessee",0500000US47125,193230,155743,14876,5250,15698,1663,47,125,2016
3,"Robertson County, Tennessee",0500000US47147,None,None,None,None,None,None,47,147,2016
4,"Williamson County, Tennessee",0500000US47187,217389,185835,12986,9100,7400,2068,47,187,2016


## Joining

In [20]:
twentysixteen = twentysixteen.set_index(['NAME', 'GEO_ID', 'Year'])

In [21]:
transp = twentysixteen.transpose()
transp.head()

NAME,"Sumner County, Tennessee","Davidson County, Tennessee","Montgomery County, Tennessee","Robertson County, Tennessee","Williamson County, Tennessee","Rutherford County, Tennessee","Wilson County, Tennessee","Maury County, Tennessee","Franklin city, Tennessee","Murfreesboro city, Tennessee","Clarksville city, Tennessee","Nashville-Davidson metropolitan government (balance), Tennessee",Tennessee,United States
GEO_ID,0500000US47165,0500000US47037,0500000US47125,0500000US47147,0500000US47187,0500000US47149,0500000US47189,0500000US47119,1600000US4727740,1600000US4751560,1600000US4715160,1600000US4752006,0400000US47,0100000US
Year,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016,2016
geo_total_series,178875,675434,193230,None,217389,305566,None,None,None,130709,148130,651790,6580953,319361956
geo_samehouse_total,153752,558893,155743,None,185835,261719,None,None,None,105491,116332,537534,5644022,272660098
geo_movedsamecounty_total,14788,61119,14876,None,12986,25395,None,None,None,14131,13091,60035,544359,26584001
geo_moveddifferentcountysamestate_total,5079,21004,5250,None,9100,10463,None,None,None,6206,3868,20333,198256,10368657
geo_moveddifferentstate_total,4155,29059,15698,None,7400,7515,None,None,None,4616,13614,28529,170867,7552536


In [24]:
numcols = list(transp.columns)
numcols
transp[numcols] = transp[numcols].astype(float)

In [25]:
data = transp

In [26]:
GNRCCounties = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                data[('Rutherford County, Tennessee', '0500000US47149')]]
data['GNRC'] = sum(GNRCCounties)
GNRCCountiesAll = [data[('Stewart County, Tennessee', '0500000US47161')],data[('Montgomery County, Tennessee', '0500000US47125')],
                   data[('Houston County, Tennessee', '0500000US47083')],data[('Humphreys County, Tennessee', '0500000US47085')],
                   data[('Dickson County, Tennessee', '0500000US47043')],data[('Cheatham County, Tennessee', '0500000US47021')],
                   data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
                   data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
                   data[('Trousdale County, Tennessee', '0500000US47169')],data[('Williamson County, Tennessee', '0500000US47187')],
                   data[('Rutherford County, Tennessee', '0500000US47149')],data[('Maury County, Tennessee', '0500000US47119')]]
data['GNRC Region'] = sum(GNRCCountiesAll)
MPOCounties = [data[('Robertson County, Tennessee', '0500000US47147')],data[('Sumner County, Tennessee', '0500000US47165')],
               data[('Davidson County, Tennessee', '0500000US47037')],data[('Wilson County, Tennessee', '0500000US47189')],
               data[('Williamson County, Tennessee', '0500000US47187')],data[('Rutherford County, Tennessee', '0500000US47149')],
               data[('Maury County, Tennessee', '0500000US47119')]]
data['MPO'] = sum(MPOCounties)
RuthInc = [data[('Eagleville city, Tennessee', '1600000US4722360')],data[('La Vergne city, Tennessee', '1600000US4741200')],
           data[('Murfreesboro city, Tennessee', '1600000US4751560')],data[('Smyrna town, Tennessee', '1600000US4769420')]]
data[('Rutherford Incorporated', 'None')] = sum(RuthInc)
data[('Rutherford Unincorporated', 'None')] = data[('Rutherford County, Tennessee', '0500000US47149')] - data[('Rutherford Incorporated', 'None')]
WilsonInc = [data[('Lebanon city, Tennessee', '1600000US4741520')],data[('Mount Juliet city, Tennessee', '1600000US4750780')],
             data[('Watertown city, Tennessee', '1600000US4778320')]]
data[('Wilson Incorporated', 'None')] = sum(WilsonInc)
data[('Wilson Unincorporated', 'None')] = data[('Wilson County, Tennessee', '0500000US47189')] - data[('Wilson Incorporated', 'None')]
CheathInc = [data[('Ashland City town, Tennessee', '1600000US4702180')],data[('Kingston Springs town, Tennessee', '1600000US4739660')],
             data[('Pegram town, Tennessee', '1600000US4757480')],data[('Pleasant View city, Tennessee', '1600000US4759560')]]
data[('Cheatham Incorporated', 'None')] = sum(CheathInc)
data[('Cheatham Unincorporated', 'None')] = data[('Cheatham County, Tennessee', '0500000US47021')] - data[('Cheatham Incorporated', 'None')]
DicksInc = [data[('Burns town, Tennessee', '1600000US4709880')],data[('Charlotte town, Tennessee', '1600000US4713080')],
            data[('Dickson city, Tennessee', '1600000US4720620')],data[('Slayden town, Tennessee', '1600000US4769080')],
            data[('Vanleer town, Tennessee', '1600000US4776860')],data[('White Bluff town, Tennessee', '1600000US4779980')]]
data[('Dickson Incorporated', 'None')] = sum(DicksInc)
data[('Dickson Unincorporated', 'None')] = data[('Dickson County, Tennessee', '0500000US47043')] - data[('Dickson Incorporated', 'None')]
HumphInc = [data[('McEwen city, Tennessee', '1600000US4744840')],data[('New Johnsonville city, Tennessee', '1600000US4752820')],
            data[('Waverly city, Tennessee', '1600000US4778560')]]
data[('Humphreys Incorporated', 'None')] = sum(HumphInc)
data[('Humphreys Unincorporated', 'None')] = data[('Humphreys County, Tennessee', '0500000US47085')] - data[('Humphreys Incorporated', 'None')]
data[('Montgomery Incorporated', 'None')] = data[('Clarksville city, Tennessee', '1600000US4715160')]
data[('Montgomery Unincorporated', 'None')] = data[('Montgomery County, Tennessee', '0500000US47125')] - data[('Montgomery Incorporated', 'None')]

In [27]:
data = data.transpose().reset_index()

In [28]:
data.head()

,NAME,GEO_ID,pop,agebysex_total_series,age_total_male,age_m_u5,age_m_5to9,age_m_10to14,age_m_15to17,age_m_18to19,age_m_20,age_m_21,age_m_22to24,age_m_25to29,age_m_30to34,age_m_35to39,age_m_40to44,age_m_45to49,age_m_50to54,age_m_55to59,age_m_60to61,age_m_62to64,age_m_65to66,age_m_67to69,age_m_70to74,age_m_75to79,age_m_80to84,age_m_85+,age_total_female,age_f_u5,age_f_5to9,age_f_10to14,age_f_15to17,age_f_18to19,age_f_20,age_f_21,age_f_22to24,age_f_25to29,age_f_30to34,age_f_35to39,age_f_40to44,age_f_45to49,age_f_50to54,age_f_55to59,age_f_60to61,age_f_62to64,age_f_65to66,age_f_67to69,age_f_70to74,age_f_75to79,age_f_80to84,age_f_85+,raceeth_total_series,raceeth_total_oneracealone,raceeth_white_alone,raceeth_blackafricanamerican_alone,raceeth_americanindianalaskanative_alone,raceeth_asian_alone,raceeth_nativehawaiianotherpacificislander_alone,raceeth_someotherrace_alone,raceeth_twoormoreraces_alone,raceeth_whitealone_nothispanicorlatino,raceeth_hispanicorlatino,hhsize_avg,units_allhousing,occupancy_total_series,occupancy_occupiedunits,occupancy_vacantunits,tenure_total_series,tenure_owneroccunits,tenure_renteroccunits,hhtype_total_series,hhtype_oneperson,hhtype_oneperson_male,hhtype_oneperson_female,hhtype_twoormore,hhtype_twoormore_family,hhtype_twoormore_family_marriedcouplefam,hhtype_twoormore_family_marriedcouplefam_ownchildrenunder18,hhtype_twoormore_family_marriedcouplefam_noownchildrenunder18,hhtype_twoormore_family_otherfam,hhtype_twoormore_family_otherfam_malenospouse,hhtype_twoormore_family_otherfam_malenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_malenospouse_noownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse,hhtype_twoormore_family_otherfam_femalenospouse_ownchildrenunder18,hhtype_twoormore_family_otherfam_femalenospouse_noownchildrenunder18,hhtype_twoormore_nonfamily,hhtype_twoormore_nonfamily_male,hhtype_twoormore_nonfamily_female,StateFIPS,GeoFIPS
0,"Cheatham County, Tennessee",0500000US47021,35912.0,35912.0,17981.0,1372.0,1473.0,1487.0,829.0,440.0,180.0,177.0,570.0,1133.0,1388.0,1787.0,1617.0,1411.0,1231.0,881.0,284.0,394.0,221.0,268.0,389.0,226.0,135.0,88.0,17931.0,1187.0,1330.0,1398.0,854.0,369.0,173.0,182.0,518.0,1201.0,1487.0,1780.0,1649.0,1372.0,1161.0,850.0,289.0,373.0,217.0,312.0,433.0,327.0,204.0,265.0,35912.0,35660.0,34783.0,532.0,135.0,63.0,17.0,130.0,252.0,34506.0,437.0,2.76,13508.0,13508.0,12878.0,630.0,12878.0,10773.0,2105.0,12878.0,2177.0,1042.0,1135.0,10701.0,10162.0,8356.0,4026.0,4330.0,1806.0,564.0,357.0,207.0,1242.0,714.0,528.0,539.0,330.0,209.0,47.0,21.0
1,"Davidson County, Tennessee",0500000US47037,569891.0,569891.0,275865.0,19466.0,18200.0,17089.0,10120.0,9254.0,4730.0,4558.0,14070.0,26542.0,24511.0,24019.0,22218.0,19891.0,16467.0,11655.0,3874.0,5310.0,3010.0,4378.0,6608.0,4941.0,2961.0,1993.0,294026.0,18347.0,17524.0,16143.0,9558.0,9399.0,4935.0,4832.0,14420.0,25950.0,23184.0,24029.0,23233.0,20978.0,17698.0,13313.0,4521.0,6409.0,4016.0,5878.0,9508.0,8301.0,5841.0,6009.0,569891.0,558652.0,381783.0,147696.0,1679.0,13275.0,403.0,13816.0,11239.0,371150.0,26091.0,2.30,252977.0,252977.0,237405.0,15572.0,237405.0,131340.0,106065.0,237405.0,79249.0,34151.0,45098.0,158156.0,138106.0,94784.0,39175.0,55609.0,43322.0,9283.0,4111.0,5172.0,34039.0,19985.0,14054.0,20050.0,11527.0,8523.0,47.0,37.0
2,"Dickson County, Tennessee",0500000US47043,43156.0,43156.0,21158.0,1538.0,1686.0,1726.0,957.0,557.0,250.0,236.0,701.0,1416.0,1594.0,1779.0,1756.0,1467.0,1361.0,1104.0,384.0,548.0,338.0,428.0,538.0,414.0,242.0,138.0,21998.0,1437.0,1584.0,1601.0,957.0,532.0,246.0,227.0,744.0,1459.0,1649.0,1753.0,1828.0,1513.0,1431.0,1129.0,381.0,556.0,322.0,505.0,693.0,579.0,462.0,410.0,43156.0,42718.0,40243.0,1978.0,172.0,116.0,5.0,204.0,438.0,40020.0,484.0,2.59,17614.0,17614.0,16473.0,1141.0,16473.0,12539.0,3934.0,16473.0,3678.0,1591.0,2087.0,12795.0,12175.0,9604.0,4322.0,5282.0,2571.0,670.0,387.0,283.0,1901.0,1162.0,739.0,620.0,384.0,236.0,47.0,43.0
3,"Houston County, Tennessee",0500000US47083,8088.

In [30]:
data.to_csv('../../pull and download formatting outputs/SF12000.csv', index = False)